In [1]:
import pandas
import numpy as np
from colour import Color

has_attention = True
dont_show_ok = False
data = pandas.read_csv("./test_dumps/main.AggloMSD-HAN-None-Word2Vec-256.1.best.csv.gz")
# print(data.head)
data["attention"] = data["attention"].apply(lambda x: np.array(eval(x)))
data["ok"] = data["prediction"] == data["label"]


In [2]:

data

,sentence,label,prediction,ok,bert_projection,attention,doc-vectors,score-negative,score-positive,urn,fp,score-prediction,probs
0,Videns eos Athenagora utrosque in amplexu cum ...,negative,negative,True,NaN,"[0.020411914214491844, 0.031495351344347, 0.05...","[-0.11335531622171402, 0.21556521952152252, 0....",0.807080,0.192920,urn:cts:latinLit:stoa0323.stoa001.digilibLT-lat1,./dataset/negative-examples/latinLit_stoa0323....,0.807080,[0.80708003 0.19292003]
1,post hunc autem ascendit Nabuchodonosor rex Ba...,negative,negative,True,NaN,"[0.05694727599620819, 0.04116060957312584, 0.0...","[-0.679416298866272, 0.6566700339317322, 0.592...",0.999924,0.000076,urn:cts:greekLit:tlg0527.tlg017.obi-lat1,./dataset/negative-examples/greekLit_tlg0527.t...,0.999924,[9.999236e-01 7.636972e-05]
2,"Ipse in primis absque peccato hominem fecit , ...",negative,negative,True,NaN,"[0.0340735986828804, 0.03552110865712166, 0.03...","[-0.4174152910709381, 0.6483216881752014, 0.32...",0.999776,0.000224,urn:cts:latinLit:stoa0121g.stoa005.opp-lat1,./dataset/negative-examples/latinLit_stoa0121g...,0.999776,[9.9977559e-01 2.2444033e-04]
3,munus ecce fictile inimus intra regiam salutis .,negative,negative,True,NaN,"[0.1295117288827896, 0.1715414673089981, 0.119...","[-0.3552809953689575, 0.2642418146133423, 0.33...",0.996918,0.003082,urn:cts:latinLit:stoa0238.stoa009.perseus-lat2,./dataset/negative-examples/latinLit_stoa0238....,0.996918,[0.99691844 0.00308162]
4,Non pol mihi quidem ;,negative,negative,True,NaN,"[0.2587473690509796, 0.2677786648273468, 0.194...","[-0.21152278780937195, 0.49693652987480164, -0...",0.983166,0.016834,urn:cts:latinLit:phi0119.phi020.perseus-lat2,./dataset/negative-examples/latinLit_phi0119.p...,0.983166,[0.98316586 0.01683417]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2737,certe ex utero sanctitatem custodisse et infan...,negative,negative,True,NaN,"[0.04119514301419258, 0.047141678631305695, 0....","[-0.49453648924827576, 0.2595881521701813, 0.2...",0.999304,0.000696,urn:cts:latinLit:stoa0104p.stoa005.opp-lat1,./dataset/negative-examples/latinLit_stoa0104p...,0.999304,[9.9930441e-01 6.9561886e-04]
2738,"Cum mora non tuta est , totis incumbere remis ...",positive,positive,True,NaN,"[0.02858753502368927, 0.0143414456397295, 0.01...","[0.20277459919452667, -0.03460775315761566, -0...",0.054845,0.945155,urn:cts:latinLit:phi0959.phi004.perseus-lat2,/home/thibault/dev/these-corpus/data/1386.xml,0.945155,[0.05484536 0.9451546 ]
2739,"nihil requirit nisi dei gratiam , solum illud ...",negative,negative,True,NaN,"[0.013131489977240562, 0.018022673204541206, 0...","[-0.4750513434410095, 0.9290459752082825, 0.68...",0.999856,0.000144,urn:cts:latinLit:stoa0022.stoa034.opp-lat1,./dataset/negative-examples/latinLit_stoa0022....,0.999856,[9.9985576e-01 1.4427115e-04]
2740,non arbitror te velle similem esse Epicureorum...,negative,negative,True,NaN,"[0.020077016204595566, 0.024200737476348877, 0...","[-0.5326380729675293, 0.5334775447845459, 0.54...",0.993972,0.006028,urn:cts:latinLit:phi0474.phi050.perseus-lat1,./dataset/negative-examples/latinLit_phi0474.p...,0.993972,[0.99397177 0.00602821]


In [4]:
from IPython.core.display import display as _display, HTML  as _HTML
data.iloc[0]

COLORS = [Color("white")] + list(Color("white").range_to(Color("red"),50))

def visualize(sentence, attention, tag="p", colors=None, display=True, label=None, is_right: bool = False,
             show_score: bool = False, use_subword=False):
    if not colors:
        colors = COLORS

    # print(attention)
    
    if use_subword:
        sentence = sentence["token_subword"]
    else:
        sentence = sentence["sentence"]
        
    html = f"<{tag}>"
    if label:
        html += f"<b style=\"{'text-decoration: line-through;' if not is_right else ''};\">{label}</b> -> "
    if show_score:
        html += f"<i>{max(attention):.2f}</i> -> "
    
    if attention is not None:
        spans = [
            f"<span style=\"background-color: {colors[min(max(0, round(s*100) - 1), 50)]}; border-radius:5px; padding:2px; display: inline-block;\">{w}</span>"
            for w, s in zip(sentence.split(" "), attention.tolist())
        ]
    else:
        spans = [
            f"<span style=\"display: inline-block;\">{w}</span>"
            for w in sentence.split(" ")
        ]
        
    html += " ".join(spans) + f"</{tag}>"

    if display:
        _display(_HTML(html))
    return html

html = "<div>"
for idx, row in data.iterrows():
    if not(row["ok"] != True and row.prediction != "negative"):
        continue
    html += "    %s\n" % visualize(
        row,
        row.get("attention") if has_attention else None,
        label=row["label"],
        is_right=row["ok"],
        use_subword=False
    )
html += "</div>"
#print(html)